In [ ]:
from sklearn.metrics import f1_score
def main():

  robert_prediction = preds
  bert_prediction = preds1
  
  electra=preds2
  final_prediction = majority_voting(robert_prediction,
                                           bert_prediction,electra)
        # Compute accuracy
  print(final_prediction)
  print("ACCURACY:", accuracy_score(real, final_prediction))

        # Save model



  a=f1_score(real, final_prediction, average='macro')
  test_prec = precision_score(real, final_prediction,average='macro');
  test_rec = recall_score(real, final_prediction,average='macro');
  print(a)
  print(test_prec)
  print(test_rec)
  return(final_prediction)

if __name__ == "__main__":
    final_prediction=main()

In [ ]:
import os
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import joblib



def majority_voting(robert_prediction, bert_prediction,electra):

    final_prediction = list()
    for robert,bert,elec in zip(robert_prediction,
                                   bert_prediction,electra):
        # Keep track of votes per class
        false = advise = mechanism = effect = intt = 0

        # Loop over all models
        predictions = [robert,bert,elec]
        for prediction in predictions:
            # Voting
            if prediction == 0:
                false += 1
            elif prediction == 1:
                advise += 1
            elif prediction == 2:
                mechanism += 1
            elif prediction == 3:
                effect += 1
            elif prediction == 4:
                intt += 1

        # Find max vote
        count_dict = {'false': false, 'advise': advise, 'mechanism': mechanism, 'effect': effect, 'int': intt}
        highest = max(count_dict.values())
        max_values = [k for k, v in count_dict.items() if v == highest]
        ensemble_prediction = []
        for max_value in max_values:
            if max_value == 'false':
                ensemble_prediction.append('false')
            elif max_value == 'advise':
                ensemble_prediction.append('advise')
            elif max_value == 'mechanism':
                ensemble_prediction.append('mechanism')
            elif max_value == 'effect':
                ensemble_prediction.append('effect')
            elif max_value == 'int':
                ensemble_prediction.append('int')

        predict = ''

        predict = ensemble_prediction[0]

        # Store max vote
        final_prediction.append(predict)
    a=[]
    for i in range(0,len(final_prediction)):
      if final_prediction[i]=='false':
        a.append(0)
      elif final_prediction[i]=='advise':
        a.append(1)
      elif final_prediction[i]=='mechanism':
        a.append(2)
      elif final_prediction[i]=='effect':
        a.append(3)
      elif final_prediction[i]=='int':
        a.append(4)
    return np.array(a)

In [ ]:
preds2 = np.concatenate(ret2[0]);
real2 = np.concatenate(ret2[1]);

print('test predictions: {}'.format(preds2));
print('real values: {}'.format(real2));

test_acc = accuracy_score(real2, preds2);
test_prec = precision_score(real2, preds2,average=None);
test_rec = recall_score(real2, preds2,average=None);
test_f1 = f1_score(real2, preds2,average=None);

print('test accuracy: {}'.format(test_acc));
print('test precision: {}'.format(test_prec));
print('test recall: {}'.format(test_rec));
print('test f1: {}'.format(test_f1));

In [ ]:
preds1 = np.concatenate(ret1[0]);
real1 = np.concatenate(ret1[1]);

print('test predictions: {}'.format(preds1));
print('real values: {}'.format(real1));

test_acc = accuracy_score(real1, preds1);
test_prec = precision_score(real1, preds1,average=None);
test_rec = recall_score(real1, preds1,average=None);
test_f1 = f1_score(real1, preds1,average=None);

print('test accuracy: {}'.format(test_acc));
print('test precision: {}'.format(test_prec));
print('test recall: {}'.format(test_rec));
print('test f1: {}'.format(test_f1));

In [ ]:
preds = np.concatenate(ret[0]);
real = np.concatenate(ret[1]);

print('test predictions: {}'.format(preds));
print('real values: {}'.format(real));

test_acc = accuracy_score(real, preds);
test_prec = precision_score(real, preds,average=None);
test_rec = recall_score(real, preds,average=None);
test_f1 = f1_score(real, preds,average=None);

print('test accuracy: {}'.format(test_acc));
print('test precision: {}'.format(test_prec));
print('test recall: {}'.format(test_rec));
print('test f1: {}'.format(test_f1));

In [ ]:
ret = test_DDI_model(model, test_data);
ret1 = test_DDI_model1(model1, test_dataa);
ret2 = test_DDI_model3(model2, test_dataaa);


In [ ]:

# load test_accumulated.tsv file
avg_test_accuracy = 0;

test_data = load_and_process_DDI_test_data('all_data_test_data.txt','test', tokenizer=tokenizer);
test_dataa = load_and_process_DDI_test_data('all_data_test_data.txt','test', tokenizer=tokenizer1);
test_dataaa = load_and_process_DDI_test_data('all_data_test_data.txt','test', tokenizer=tokenizer2);


In [ ]:

def test_DDI_model3(model, data, batch_size=12):
    model.load_state_dict(torch.load('bioelectra-base-discriminator-pubmed.pt'));

    model.eval();

    ds = data;

    test_dataloader = DataLoader(
        ds,
        sampler=SequentialSampler(ds),
        batch_size=batch_size,
    );

    preds_list, real_labels_list = [], [];

    for batch_nbr, batch in enumerate(test_dataloader):
        b_input_ids = batch[0].to(device2);
        b_input_mask = batch[1].to(device2);
        b_labels = batch[2].to(device2);

        with torch.no_grad():
            output = model(
                b_input_ids,
                token_type_ids=None,
                attention_mask=b_input_mask
            );
        loss = output.loss;
        preds = torch.argmax(output.logits, dim=1).flatten()
        logits = preds.detach().cpu().numpy();
        label_ids = b_labels.to('cpu').numpy();
        

        preds_list.append(logits);
        real_labels_list.append(label_ids);

        print('finished batch {}'.format(batch_nbr));

    ret = (
        preds_list,
        real_labels_list
    );
    
    return ret;

In [ ]:

def test_DDI_model1(model, data, batch_size=12):
    model.load_state_dict(torch.load('biobert.pt'));

    model.eval();

    ds = data;

    test_dataloader = DataLoader(
        ds,
        sampler=SequentialSampler(ds),
        batch_size=batch_size,
    );

    preds_list, real_labels_list = [], [];

    for batch_nbr, batch in enumerate(test_dataloader):
        b_input_ids = batch[0].to(device1);
        b_input_mask = batch[1].to(device1);
        b_labels = batch[2].to(device1);

        with torch.no_grad():
            output = model(
                b_input_ids,
                token_type_ids=None,
                attention_mask=b_input_mask
            );
        loss = output.loss;
        preds = torch.argmax(output.logits, dim=1).flatten()
        logits = preds.detach().cpu().numpy();
        label_ids = b_labels.to('cpu').numpy();
        

        preds_list.append(logits);
        real_labels_list.append(label_ids);

        print('finished batch {}'.format(batch_nbr));

    ret = (
        preds_list,
        real_labels_list
    );
    
    return ret;

In [ ]:

def test_DDI_model(model, data, batch_size=12):
    model.load_state_dict(torch.load('biomed_roberta_base.pt'));

    model.eval();

    ds = data;

    test_dataloader = DataLoader(
        ds,
        sampler=SequentialSampler(ds),
        batch_size=batch_size,
    );

    preds_list, real_labels_list = [], [];

    for batch_nbr, batch in enumerate(test_dataloader):
        b_input_ids = batch[0].to(device);
        b_input_mask = batch[1].to(device);
        b_labels = batch[2].to(device);

        with torch.no_grad():
            output = model(
                b_input_ids,
                token_type_ids=None,
                attention_mask=b_input_mask
            );
        loss = output.loss;
        preds = torch.argmax(output.logits, dim=1).flatten()
        logits = preds.detach().cpu().numpy();
        label_ids = b_labels.to('cpu').numpy();
        

        preds_list.append(logits);
        real_labels_list.append(label_ids);

        print('finished batch {}'.format(batch_nbr));

    ret = (
        preds_list,
        real_labels_list
    );
    
    return ret;

In [ ]:
from IPython.display import display;
def load_and_process_DDI_test_data(filepath,set_type, tokenizer, maxlen=512):
    
    relation_labels=['false', 'mechanism', 'effect', 'advise', 'int']
    print( "Input File Reading")
    fp = open(filepath, 'r')
    samples = fp.read().strip().split('\n\n')
    sent_lengths   = []		#1-d array
    sent_contents  = []		#2-d array [[w1,w2,....] ...]
    labels    = []		#1-d array
    entity1_list   = []		#2-d array [[e1,e1_t] [e1,e1_t]...]
    entity2_list   = []		#2-d array [[e1,e1_t] [e1,e1_t]...]
    examples = []
    i=0
    for sample in samples:
      guid = "%s-%s" % (set_type, i)
      sent, entities, relation = sample.strip().split('\n')

      e1, e1_t, e2, e2_t = entities.split('\t') 
      sent_contents.append(sent.lower())
      label = relation_labels.index(relation) 
      examples.append(InputExample(guid=guid, text_a=sent.lower(), label=label))
      entity1_list.append([e1, e1_t])
      entity2_list.append([e2, e2_t])
      print(label)
      labels.append(label)
      i = i+1



    input_ids = [];
    attention_masks = [];
    
    for sent in sent_contents:
        encoded_dict = tokenizer.encode_plus(
            sent,
            add_special_tokens=True,
            max_length=512,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        );

        input_ids.append(encoded_dict['input_ids']);
        attention_masks.append(encoded_dict['attention_mask']);

    # convert lists into tensors
    input_ids = torch.cat(input_ids, dim=0);
    attention_masks = torch.cat(attention_masks, dim=0);
    labels_tensor = torch.tensor(labels);

    dataset = TensorDataset(input_ids, attention_masks, labels_tensor);


    return dataset;

In [ ]:

data = load_and_process_DDI_train_data('all_data_train_data.txt','train' ,tokenizer=tokenizer, maxlen=128);

dataa = load_and_process_DDI_train_data('all_data_train_data.txt','train' ,tokenizer=tokenizer1, maxlen=128);

dataaa = load_and_process_DDI_train_data('all_data_train_data.txt','train' ,tokenizer=tokenizer2, maxlen=256);



In [ ]:

tokenizer2 = ElectraTokenizer.from_pretrained('kamalkraj/bioelectra-base-discriminator-pubmed',do_lower_case=True)
model2 = ElectraForSequenceClassification.from_pretrained(
    'kamalkraj/bioelectra-base-discriminator-pubmed',
    num_labels=5,
    output_attentions=False,
    output_hidden_states=False
);

device2 = torch.device('cuda');
model2.cuda();

epochs = 2;

In [ ]:

tokenizer1 = BertTokenizer.from_pretrained('monologg/biobert_v1.1_pubmed');

model1 = BertForSequenceClassification.from_pretrained(
    'monologg/biobert_v1.1_pubmed',
    num_labels=5,
    output_attentions=False,
    output_hidden_states=False
);

device1 = torch.device('cuda');
model1.cuda();

epochs = 2;

In [ ]:

#tokenizer = RobertaTokenizer.from_pretrained('simonlevine/bioclinical-roberta-long');
tokenizer = RobertaTokenizer.from_pretrained('simonlevine/biomed_roberta_base-4096')
model = RobertaForSequenceClassification.from_pretrained(
    'simonlevine/biomed_roberta_base-4096',
    num_labels=5,
    output_attentions=False,
    output_hidden_states=False
);

device = torch.device('cuda');
model.cuda();

epochs = 2;

In [ ]:
def load_and_process_DDI_train_data(filepath,set_type, tokenizer, maxlen=512, train_percentage=0.7):
    # load dataset

    relation_labels=['false', 'mechanism', 'effect', 'advise', 'int']
    print( "Input File Reading")
    fp = open(filepath, 'r')
    samples = fp.read().strip().split('\n\n')
    sent_lengths   = []		#1-d array
    sent_contents  = []		#2-d array [[w1,w2,....] ...]
    labels    = []		#1-d array
    entity1_list   = []		#2-d array [[e1,e1_t] [e1,e1_t]...]
    entity2_list   = []		#2-d array [[e1,e1_t] [e1,e1_t]...]
    examples = []
    i=0
    for sample in samples:
      guid = "%s-%s" % (set_type, i)
      sent, entities, relation = sample.strip().split('\n')

      e1, e1_t, e2, e2_t = entities.split('\t') 
      sent_contents.append(sent.lower())
      label = relation_labels.index(relation) 
      examples.append(InputExample(guid=guid, text_a=sent.lower(), label=label))
      entity1_list.append([e1, e1_t])
      entity2_list.append([e2, e2_t])
      print(label)
      labels.append(label)
      i = i+1



    input_ids = [];
    attention_masks = [];
    
    for sent in sent_contents:
        encoded_dict = tokenizer.encode_plus(
            sent,
            add_special_tokens=True,
            max_length=maxlen,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        );

        input_ids.append(encoded_dict['input_ids']);
        attention_masks.append(encoded_dict['attention_mask']);
        

    # convert lists into tensors
    input_ids = torch.cat(input_ids, dim=0);
    attention_masks = torch.cat(attention_masks, dim=0);
    labels_tensor = torch.tensor(labels);

    dataset = TensorDataset(input_ids, attention_masks, labels_tensor);

    train_size = int(train_percentage * len(dataset));
    val_size = len(dataset) - train_size;

    train_dataset, val_dataset = random_split(dataset, [train_size, val_size]);

    return (train_dataset, val_dataset);


In [ ]:
!pip install sentencepiece

!pip install transformers
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import numpy as np;
import pandas as pd;

import torch;

from transformers import BertTokenizer, BertForSequenceClassification, AdamW, BertConfig;
from torch.utils.data import TensorDataset, random_split, DataLoader, RandomSampler, SequentialSampler;
from transformers import get_linear_schedule_with_warmup;

from transformers import (WEIGHTS_NAME, BertConfig, BertForSequenceClassification, BertTokenizer, BertModel, BertForTokenClassification,
                                  XLMConfig, XLMForSequenceClassification, XLMTokenizer, 
                                  XLNetConfig, XLNetForSequenceClassification, XLNetTokenizer,
                                  RobertaConfig, RobertaForSequenceClassification, RobertaTokenizer,
                                  AlbertConfig, AlbertForSequenceClassification, AlbertTokenizer, AlbertModel,AlbertForMaskedLM,
                                  AutoTokenizer, AutoModelForTokenClassification, AutoConfig,
                           XLMRobertaTokenizer,XLMRobertaForSequenceClassification,
                          RobertaConfig, RobertaForSequenceClassification, RobertaTokenizer,
                          ElectraTokenizer,ElectraForSequenceClassification
                          )
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score;
from transformers import AlbertTokenizer, AlbertForSequenceClassification, AlbertConfig, AdamW, DistilBertForSequenceClassification, DistilBertTokenizer, DistilBertConfig


import os
import re
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

import gc
    
gc.collect()
import torch
torch.cuda.empty_cache()

import torch

if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

import os
import csv
import copy
import json
import logging

import torch
from torch.utils.data import TensorDataset


logger = logging.getLogger(__name__)

class InputExample(object):
    """
    A single training/test example for simple sequence classification.
    Args:
        guid: Unique id for the example.
        text_a: string. The untokenized text of the first sequence. For single
        sequence tasks, only this sequence must be specified.
        label: (Optional) string. The label of the example. This should be
        specified for train and dev examples, but not for test examples.
    """

    def __init__(self, guid, text_a, label):
        self.guid = guid
        self.text_a = text_a
        self.label = label

    def __repr__(self):
        return str(self.to_json_string())

    def to_dict(self):
        """Serializes this instance to a Python dictionary."""
        output = copy.deepcopy(self.__dict__)
        return output

    def to_json_string(self):
        """Serializes this instance to a JSON string."""
        return json.dumps(self.to_dict(), indent=2, sort_keys=True) + "\n"


from transformers import BertTokenizer

# Load the BERT tokenizer

#tokenizer = RobertaTokenizer.from_pretrained('simonlevine/bioclinical-roberta-long')
tokenizer = RobertaTokenizer.from_pretrained('simonlevine/biomed_roberta_base-4096')

tokenizer1 = BertTokenizer.from_pretrained('monologg/biobert_v1.1_pubmed')


tokenizer2 = ElectraTokenizer.from_pretrained('kamalkraj/bioelectra-base-discriminator-pubmed',do_lower_case=True)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/
!ls
